## All imports needed for detectron working

In [1]:
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
import cv2
import random
import numpy as np
import os
import time

ImportError: libshm.so: cannot open shared object file: No such file or directory

## This block is to define the dataset, the image you want to test and define the model.

In [2]:
## Change this to adapt the path where you have installed detectron2 framework
DETECTRON2_ROOT = "/home/user"

register_coco_instances("Speed", {}, DETECTRON2_ROOT + "/detectron2/datasets/custom_coco/annotations/new_speed-30.json", DETECTRON2_ROOT + "/detectron2/datasets/custom_coco/images")

img_path = DETECTRON2_ROOT + "/detectron2/datasets/custom_coco/images/20210429_163839.jpg"

Speed_metadata = MetadataCatalog.get("Speed")
dataset_dicts = DatasetCatalog.get("Speed")

cfg = get_cfg()
cfg.merge_from_file(
    "configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)

cfg.MODEL.WEIGHTS = DETECTRON2_ROOT + "/detectron2/output/model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

NameError: name 'register_coco_instances' is not defined

## This block will read the image and draw the class on the relevant objects.

In [3]:
im = cv2.imread(img_path)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
               metadata=Speed_metadata, 
               scale=0.8, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
img = v.get_image()[:, :, ::-1]
print(img.shape)
cv2.imshow("test",  cv2.resize(img,  ( int( img.shape[0] /2), int(img.shape[1] /2)) ) )

times = []
for i in range(20):
    start_time = time.time()
    outputs = predictor(im)
    delta = time.time() - start_time
    times.append(delta)
mean_delta = np.array(times).mean()
fps = 1 / mean_delta
print("Average(sec):{:.2f},fps:{:.2f}".format(mean_delta, fps))

#(this is necessary to avoid Python kernel form crashing) 
cv2.waitKey(0)  
#closing all open windows  
cv2.destroyAllWindows()

/home/cristal/.local/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/home/cristal/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


(3225, 2419, 3)
Average(sec):0.46,fps:2.18
